# Lecture 13: GNIS

In [ ]:
import numpy as np
import pandas as pd

---

# Scalar Functions and Query Plans

In [ ]:
## we'll use the Lahman baseball database in our examples today.
## replace the database connection with a database of your own!
%reload_ext sql
%sql postgresql://localhost:5432/baseball
%config SqlMagic.displaylimit = 20

In [ ]:
%%sql
WITH yearnum AS
  (SELECT yearid, (yearid % 100) as year
     FROM batting
  )
SELECT yearid, CONCAT('''', LPAD(year::text, 2, '0')) as year
  FROM yearnum
 LIMIT 5;

Let's analyze the below query (we've flattened it for convenience):

In [ ]:
%%sql
EXPLAIN (VERBOSE true)
SELECT yearid,
       CONCAT('''', LPAD((yearid % 100)::text, 2, '0')) AS year
FROM batting;


What if scalar functions mention multiple tables?

The below query computes an arbitrary statistic for pitchers:
* 1 point for every strikeout they throw as pitcher
* –1 for every point they themselves struck out as batter

If the notebook-like output is hard to read, try out the query in `psql`. Note that notebooks don't preserve whitespace when displaying dataframes.

In [ ]:
%%sql
EXPLAIN (VERBOSE true)
SELECT p.playerid, p.so - b.so
  FROM pitching p
  INNER JOIN batting b
  ON p.playerid=b.playerid;

### Window Functions

In [ ]:
%%sql
SELECT namefirst, namelast, yearid, HR,
       rank() OVER (ORDER BY HR DESC),
       avg(HR)    OVER (PARTITION BY b.playerid ORDER BY yearid ROWS 3 PRECEDING) as avg_3yr,
       lag(HR, 7) OVER (PARTITION BY b.playerid ORDER BY yearid) as previous,
       lag(HR, 2) OVER (PARTITION BY b.playerid ORDER BY yearid) as lag2
FROM batting b, master m
WHERE m.playerid = b.playerid
   AND (namelast = 'Bonds' or namelast = 'Ruth')
ORDER BY HR DESC
LIMIT 10;

### Inverse Distribution Window Functions

In [ ]:
%%sql
SELECT MIN(HR),
       percentile_cont(0.25) WITHIN GROUP (ORDER BY HR) AS p25,
       percentile_cont(0.50) WITHIN GROUP (ORDER BY HR) AS median,
       percentile_cont(0.75) WITHIN GROUP (ORDER BY HR) AS p75,
       percentile_cont(0.99) WITHIN GROUP (ORDER BY HR) AS p99,
       MAX(HR),
       AVG(HR)
FROM batting;

In [ ]:
%%sql
SELECT HR, COUNT(*) FROM batting GROUP BY HR ORDER BY HR;

### Hypothetical-Set Window Functions

In [ ]:
hrs = 4 # hypothetically, four home runs

In [ ]:
%%sql
SELECT {{hrs}} as hypothetical,
       rank({{hrs}}) WITHIN GROUP (ORDER BY HR DESC),
       dense_rank({{hrs}}) WITHIN GROUP (ORDER BY HR DESC),
       percent_rank({{hrs}}) WITHIN GROUP (ORDER BY HR DESC) * 100 AS pct_rank,
       cume_dist({{hrs}}) WITHIN GROUP (ORDER BY HR)
FROM batting
LIMIT 10;

Without `jupysql` variable substituion

In [ ]:
%%sql
SELECT 4 as hypothetical,
       rank(4) WITHIN GROUP (ORDER BY HR DESC),
       dense_rank(4) WITHIN GROUP (ORDER BY HR DESC),
       percent_rank(4) WITHIN GROUP (ORDER BY HR DESC) * 100 AS pct_rank,
       cume_dist(4) WITHIN GROUP (ORDER BY HR)
FROM batting
LIMIT 10;

# GNIS

This notebook transforms the existing [Geographics Names Information Systems (GNIS)](https://www.usgs.gov/core-science-systems/ngp/board-on-geographic-names/download-gnis-data) national zip file.

We have provided a subset of the sql database for you in `data/national.sql`.

If you'd like to make your own version of this database, see the end of this notebook. Note: Because of its size, we don't recommend building the GNIS SQL database from scratch on DataHub.


In [ ]:
!psql -h localhost -d gnis -c 'SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE datname = current_database() AND pid <> pg_backend_pid();'
!psql -h localhost -c 'DROP DATABASE IF EXISTS gnis'
!psql -h localhost -c 'CREATE DATABASE gnis' 
!psql -h localhost -d gnis -f data/gnis.sql

In [ ]:
%reload_ext sql
%sql postgresql://localhost:5432/gnis
%config SqlMagic.displaylimit = 15

* View schema in `psql`
* View some rows below

In [ ]:
%sql SELECT COUNT(*) FROM national;

In [ ]:
%sql SELECT * FROM national WHERE county_name = 'Alameda';

In [ ]:
%%sql
SELECT *
FROM national TABLESAMPLE BERNOULLI(10);

# Numerical Granularity

In [ ]:
%sql SELECT elev_in_m FROM National LIMIT 2;

In [ ]:
%%sql
SELECT elev_in_m, 
    (elev_in_m / 100) :: INTEGER AS quantized,
    ((elev_in_m / 100) :: INTEGER) * 100 AS round_to_100,
    SUBSTRING(elev_in_m :: TEXT, 1, 2),
    CONCAT(SUBSTRING(elev_in_m :: TEXT, 1, 2), '00') AS substring2
FROM National
LIMIT 5;

In [ ]:
%%sql
/* does not run in jupysql b/c of <<. copy into psql  */
SELECT elev_in_m,
    (16::INTEGER::BIT(12)) AS bit12, 
    (16::INTEGER::BIT(12)) << 3
FROM national
LIMIT 5;

In [ ]:
%%sql
EXPLAIN (verbose)
WITH shifts AS (
SELECT elev_in_m, 
       (elev_in_m::integer::bit(12)) AS bit12, 
       (elev_in_m::integer::bit(12) >> 8) AS rightshifted, 
       ((elev_in_m::integer::bit(12) >> 8) << 8)::integer AS round_to_256,
       ((elev_in_m::integer::bit(12) >> 8) << 8)::integer % 256 AS test
  FROM national
)
SELECT COUNT(DISTINCT elev_in_m) AS elevation_meters_count,
       COUNT(DISTINCT bit12) AS bit12_count,
       COUNT(DISTINCT rightshifted) AS rightshift_count,
       COUNT(DISTINCT round_to_256) AS rounded_count
  FROM shifts;

# Demo 1: Roll-up / Drill-down Practice

Let's start with county-level data on elevations:

In [ ]:
%%sql
SELECT state_numeric, county_numeric,
       avg(elev_in_m),
       stddev(elev_in_m), count(*)
FROM national TABLESAMPLE BERNOULLI(10)
GROUP BY state_numeric, county_numeric;

**Roll up** to state level.
* We save the view as `state_elevations` for later...

In [ ]:
%%sql
DROP VIEW IF EXISTS state_elevations;

CREATE VIEW state_elevations AS
SELECT state_numeric,
       avg(elev_in_m),
       stddev(elev_in_m), count(*)
FROM national 
GROUP BY state_numeric
;

In [ ]:
%sql SELECT * FROM state_elevations;

**Drill down** to include feature class.

In [ ]:
%%sql
SELECT state_numeric, feature_class,
       avg(elev_in_m),
       stddev(elev_in_m), count(*)
FROM national TABLESAMPLE Bernoulli(10)
GROUP BY state_numeric, feature_class
ORDER BY count(*) DESC;

# Demo 2: Connections to Statistics

## Roll up with marginal distributions

In [ ]:
%%sql
SELECT state_numeric,
       AVG(elev_in_m),
       STDDEV(elev_in_m), COUNT(*),
       SUM(COUNT(*)) OVER () AS total,
       COUNT(*)/SUM(COUNT(*)) OVER () AS marginal
FROM national TABLESAMPLE Bernoulli(.07)
GROUP BY state_numeric;

In [ ]:
%%sql
SELECT COUNT(DISTINCT county_numeric) FROM national;

## Drill down with normally-distributed elevations:

Start with the `state_elevations` view from earlier:

In [ ]:
%sql SELECT * FROM state_elevations;

The `fips_counties` relation has all counties, including those not in `national`:

In [ ]:
%sql SELECT * FROM fips_counties LIMIT 10;

If we wanted to **drill down** to the FIPS counties, we'd need to simulate an elevation for those counties that don't exist in `national`.

Here's the first step in that process, which creates a simulated value for *every* county in `fips_counties`.
* The value is simulated from a normal distribution using that state's elevation statistics (average, standard deviation).
* Just like a Python package, we would need to import `tablefunc` in order to use the `normal_rand` function.

In [ ]:
%sql CREATE EXTENSION IF NOT EXISTS tablefunc;

In [ ]:
%%sql
WITH state_cty AS
(SELECT s.state_numeric, f.fips as county_numeric, s.avg, s.stddev, s.count
  FROM state_elevations s, fips_counties f
  WHERE s.state_numeric = f.state_numeric
)
SELECT s.*,
       n.n AS elev_in_m,
       true as elev_in_m_sim -- user-facing flag
  FROM state_cty s,
       LATERAL normal_rand(CAST(s.count AS INTEGER), s.avg, s.stddev) AS n
LIMIT 10;

# Assembling an Explicit Hierarchy

In [ ]:
## we'll use the Lahman baseball database in our initial examples today.
## replace the database connection with a database of your own!
%reload_ext sql
%sql postgresql://localhost:5432/baseball

Two relations have the pieces of the hierarchy we want:

In [ ]:
%sql SELECT * FROM Appearances WHERE yearid > 1970 LIMIT 2;

In [ ]:
%sql SELECT * FROM Teams LIMIT 1;

Let's join these two to make our hierarchy! Which way should we make this?

In [ ]:
%%sql
SELECT a.playerid, a.teamid, t.divid, a.*
  FROM Appearances a
  NATURAL JOIN Teams t
WHERE a.yearid = 2015
LIMIT 100;

In [ ]:
%%sql
CREATE OR REPLACE VIEW bball_tree AS
SELECT DISTINCT a.playerid, a.teamid, t.divid, a.lgid, a.yearid
  FROM Appearances a NATURAL JOIN Teams t;

### Revisiting the Home Run Query

Recall our old home run query:

In [ ]:
%%sql
SELECT namefirst, namelast, yearid,
       MIN(hr), MAX(hr), AVG(hr), STDDEV(hr), SUM(hr)
FROM batting b, people p
WHERE b.playerid = p.playerid
GROUP BY namelast, namefirst, yearid
ORDER BY max DESC
LIMIT 10;

Set up for roll up/drill down on `bball_tree` hierarchy.
* Join each (raw) person with the associated `bball_tree` entry by `(playerid, yearid)` in a CTE
* Use this result for roll-up and drill-down.

<br/><br/><br/><br/><br/>
<br/><br/><br/><br/><br/>
<br/><br/><br/><br/><br/>
<br/><br/><br/><br/><br/>

In [ ]:
%%sql
WITH batting_tree AS
(
    SELECT b.*, t.divid
    FROM batting b, bball_tree t
    WHERE b.playerid = t.playerid
      AND b.yearid = t.yearid
)
SELECT namefirst, namelast,
       bt.teamid, bt.lgid, bt.divid, bt.yearid,
       MIN(hr), MAX(hr), AVG(hr), STDDEV(hr), SUM(hr)
FROM batting_tree bt, people p
WHERE bt.playerid = p.playerid
GROUP BY bt.playerid, bt.teamid, bt.lgid, bt.divid, bt.yearid, namelast, namefirst
ORDER BY max DESC
LIMIT 10;

# [Extra] Load in the database from scratch

We download the database, unzip it, load it into pandas, then export to a new database via `jupysql` cell magic.

**CAUTION**: This may crash your DataHub instance. The file is pretty big....

The direct zip download of this file is [here](https://geonames.usgs.gov/docs/stategaz/NationalFile.zip).

In [ ]:
# first download and unzip the data
!mkdir -p data
!wget https://geonames.usgs.gov/docs/stategaz/NationalFile.zip -P data/
!unzip -u data/NationalFile.zip -d data/

In [ ]:
import os
fname = os.path.join("data", "NationalFile_20210825.txt")
fname

In [ ]:
!du -h {fname} # big file

In [ ]:
!head -c 1024 {fname}

In [ ]:
# next, load it into pandas
import pandas as pd

national = pd.read_csv("data/NationalFile_20210825.txt", delimiter="|")
national.head(2)

In [ ]:
national = national.rename(columns=dict([(col, col.lower().strip()) for col in national.columns]))
national.head(2)

Next, get a table sample in pandas.

In [ ]:
import numpy as np

p = 0.005 # fraction, not percentage

np.random.seed(42)
national['keep_bool'] = np.random.random(len(national)) < p
national['keep_bool'].value_counts()

In [ ]:
national = national[national['keep_bool']].drop(columns=['keep_bool'])
national

Now, export to SQL

In [ ]:
!psql -h localhost -d gnis -c 'SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE datname = current_database() AND pid <> pg_backend_pid();'
!psql -h localhost -c 'DROP DATABASE IF EXISTS gnis'
!psql -h localhost -c 'CREATE DATABASE gnis' 

In [ ]:
%reload_ext sql
%sql postgresql://127.0.0.1:5432/gnis

In [ ]:
%sql --persist-replace national

In [ ]:
%sql ALTER TABLE national DROP COLUMN index;

Now, export to file with `pgdump`

In [ ]:
!pg_dump -h localhost --encoding utf8 gnis -f data/gnis.sql 

Finally, run the beginning of this notebook again

In [ ]:
!du -h data/gnis.sql

## FIPS

Federal Information Processing System (FIPS) Codes for States and Counties

Manually download the file from this link (https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt) and save it in `data/`.
* `wget` does not work here; likely the FCC website only accepts HTTPS connections to deter from server attacks.

In [ ]:
!wget https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt -P data/

In [ ]:
import pandas as pd
import re

In [ ]:
with open('data/fips.txt', 'r') as f:
    lines = f.readlines()

In [ ]:
COUNTY_STARTS = 69
OFFSET = 3 # the start of the data itself, after headers

In [ ]:
re.match('\s+(\d+)\s+(\w+)', lines[COUNTY_STARTS+3]).groups()

In [ ]:
splits = [re.match('\s+(\d+)\s+(.*)', line).groups()
          for line in 
          lines[COUNTY_STARTS+OFFSET:]]
splits[0]

**For later**: There is a significant discrepancy between the number of counties created and the number of lines remaining in our dataset. We encourage you to investigate this!

In [ ]:
len(lines), len(splits)

> FIPS codes are numbers which uniquely identify geographic areas.  The number of 
digits in FIPS codes vary depending on the level of geography.  State-level FIPS
codes have two digits, county-level FIPS codes have five digits of which the 
first two are the FIPS code of the state to which the county belongs.  When 
using the list below to look up county FIPS codes, it is advisable to first look
up the FIPS code for the state to which the county belongs.  This will help you
identify the right section of the list while scrolling down, which can be
important since there are over 3000 counties and county-equivalents (e.g.
independent cities, parishes, boroughs) in the United States.

In [ ]:
fips_counties = pd.DataFrame(data=splits, columns=['fips', 'county'])
fips_counties['state_numeric'] = fips_counties['fips'].str[:2].astype(int)
fips_counties['fips'] = fips_counties['fips'].astype(int)
fips_counties = fips_counties.set_index('fips')
fips_counties

In [ ]:
%reload_ext sql
%sql postgresql://127.0.0.1:5432/gnis

In [ ]:
%sql --persist-replace fips_counties

Now, export to file with `pgdump`. This exports both `national` and `fips_counties` relations to the same `gnis.sql` database dump.

In [ ]:
!pg_dump -h localhost --encoding utf8 gnis -f data/gnis.sql 

Finally, run the beginning of this notebook again

In [ ]:
!du -h data/gnis.sql